In [ ]:
from tqdm import tqdm
import numpy as np

## Part 1

In [ ]:
def push_col_north(col):
    col_length = len(col)
    next_available_spot = 0
    for xdx, type in enumerate(col):
        if col[xdx] == 'O' and next_available_spot < xdx:
            col[xdx] = '.'
            col[next_available_spot] = 'O'
        while next_available_spot<col_length-1 and col[next_available_spot] != '.':
            next_available_spot +=1
    return col

test1 = np.array([['O', '.', '#', '.'], 
         ['O', 'O', '.', '.'],
         ['O', 'O', 'O', 'O']])
n_cols = len(test1[0])

for ydx in range(n_cols):
    print(test1[ydx, :])
    test1[ydx,:] = push_col_north(test1[ydx,:])

In [ ]:
#North
def coord_North_free_spots_in_dir(total_map: list([]), coord_last_row_yx):
    total_map_row_above_x = coord_last_row_yx[0]
    
    
    while total_map_row_above_x>0 and total_map[total_map_row_above_x -1][coord_last_row_yx[1]] == '.':
        total_map_row_above_x -=1
        if not total_map_row_above_x:
            break
    return total_map_row_above_x

test1 = [['O', '.', '#', '.'], 
         ['O', 'O', '.', '.'],
         ['O', 'O', 'O', 'O']]
for idx in range(4):
    print(coord_North_free_spots_in_dir(test1, [2,idx]))

In [ ]:
#West
def coord_West_free_spots_in_dir(total_map: list([]), coord_last_row_yx):
    total_map_cols_before_y = coord_last_row_yx[1]
    
    
    while total_map_cols_before_y>0 and total_map[coord_last_row_yx[0]][total_map_cols_before_y -1] == '.':
        total_map_cols_before_y -=1
        if not total_map_cols_before_y:
            break
    return total_map_cols_before_y

test1 = [['O', '.', 'O', '.'], 
         ['.', 'O', 'O', '.'],
         ['.', '.', 'O', 'O']]
for idx in range(3):
    print(coord_West_free_spots_in_dir(test1, [idx,2]))

In [ ]:
#South
def coord_South_free_spots_in_dir(total_map: list([]), coord_last_row_yx):
    n_rows = len(total_map)
    ydx = None
    for ydx in range(coord_last_row_yx[0]+1, n_rows):
        if total_map[ydx][coord_last_row_yx[1]] != '.':
            ydx = ydx-1
            break
    return ydx or coord_last_row_yx[0]

test1 = [['O', 'O', 'O', 'O'], 
         ['O', '.', '.', '.'],
         ['O', 'O', '.', 'O']]
for idx in range(4):
    print(coord_South_free_spots_in_dir(test1, [0,idx]))

In [ ]:
#East
def coord_East_free_spots_in_dir(total_map: list([]), coord_last_row_yx):
    n_cols = len(total_map[0])
    
    xdx = None
    for xdx in range(coord_last_row_yx[1]+1, n_cols):
        if total_map[coord_last_row_yx[0]][xdx] != '.':
            xdx = xdx-1
            break
    return xdx or coord_last_row_yx[1]

test1 = [['O', '.', '#', '.'], 
         ['O', 'O', '.', '.'],
         ['O', '.', '.', '.'],
         ['O', '.', '.', 'O']]
for idx in range(3):
    print(coord_East_free_spots_in_dir(test1, [idx,0]))
print(coord_East_free_spots_in_dir(test1, [3,3]))

In [ ]:
total_map = []
weight_sum = 0
# generate maps 
with open('Day14_input.txt') as f:
    for line in f:
        line = list(line.strip('\n'))
        total_map.append(line)
        for xdx, block in enumerate(total_map[-1]):
            if block == 'O':
                new_row = coord_North_free_spots_in_dir(total_map, [len(total_map)-1, xdx])
                total_map[-1][xdx] = '.'
                total_map[new_row][xdx] = 'O'
                weight_sum += 100 - new_row
                
print(weight_sum)

## Part 2

In [ ]:
def count_points(matrix):
    total_depth = len(matrix)
    points = 0
    for ydx, line in enumerate(matrix):
        for block in line:
            if block =='O':
                points += total_depth - ydx
    return points

print(count_points(total_map))

In [ ]:
total_map = []
weight_sum = 0
# generate maps 
with open('Day14_input.txt') as f:
    for line in f:
        line = list(line.strip('\n'))
        total_map.append(line)


n_rows = len(total_map)
n_cols = len(total_map[0])
n_spins = pow(10,9)
for spin in tqdm(range(n_spins)):
    #North
    for ydx in range(n_rows):
        for xdx, block in enumerate(total_map[ydx]):
            if block == 'O':
                new_row = coord_North_free_spots_in_dir(total_map,[ydx, xdx])
                total_map[ydx][xdx] = '.'
                total_map[new_row][xdx] = 'O'
                                                                 
    #West
    for xdx in range(n_cols):
        West_col = [total_map[y][xdx] for y in range(n_rows)]
        for ydx, block in enumerate(West_col):
            if block == 'O':
                new_row = coord_West_free_spots_in_dir(total_map,[ydx, xdx])
                total_map[ydx][xdx] = '.'
                total_map[new_row][xdx] = 'O'                                                                 
                                                                 
    #South
    for ydx in range(n_rows-1, -1, -1):
        for xdx, block in enumerate(total_map[ydx]):
            if block == 'O':
                new_row = coord_South_free_spots_in_dir(total_map,[ydx, xdx])
                total_map[ydx][xdx] = '.'
                total_map[new_row][xdx] = 'O'    

    #East
    for xdx in range(n_cols-1, -1, -1):
        East_col = [total_map[y][xdx] for y in range(n_rows)]
        for ydx, block in enumerate(East_col):
            if block == 'O':
                new_row = coord_East_free_spots_in_dir(total_map,[ydx, xdx])
                total_map[ydx][xdx] = '.'
                total_map[new_row][xdx] = 'O'                
                
                
print(count_points(total_map))